# Geocode Singapore Addresses

In [1]:
import pandas as pd
import glob
import geopandas as gpd
from geopy.geocoders import MapBox
from geopy.extra.rate_limiter import RateLimiter

##### Read in CSV

In [2]:
resale_price_all = glob.glob("./*.csv")

li = []

for filename in resale_price_all:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
resale_df = pd.concat(li, axis=0, ignore_index=True)
resale_df['ADDRESS'] = resale_df['block'].astype(str) + ' ' + resale_df['street_name'] + ',' + ' Singapore'   
resale_df.info()
resale_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831336 entries, 0 to 831335
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                831336 non-null  object 
 1   town                 831336 non-null  object 
 2   flat_type            831336 non-null  object 
 3   block                831336 non-null  object 
 4   street_name          831336 non-null  object 
 5   storey_range         831336 non-null  object 
 6   floor_area_sqm       831336 non-null  float64
 7   flat_model           831336 non-null  object 
 8   lease_commence_date  831336 non-null  int64  
 9   resale_price         831336 non-null  float64
 10  remaining_lease      122282 non-null  object 
 11  ADDRESS              831336 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 76.1+ MB


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,ADDRESS
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN,"172 ANG MO KIO AVE 4, Singapore"
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN,"510 ANG MO KIO AVE 8, Singapore"
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN,"610 ANG MO KIO AVE 4, Singapore"
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN,"474 ANG MO KIO AVE 10, Singapore"
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN,"604 ANG MO KIO AVE 5, Singapore"
...,...,...,...,...,...,...,...,...,...,...,...,...
831331,2016-12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,488000.0,82,"297 YISHUN ST 20, Singapore"
831332,2016-12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,455000.0,69,"838 YISHUN ST 81, Singapore"
831333,2016-12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,778000.0,74,"664 YISHUN AVE 4, Singapore"
831334,2016-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,575000.0,70,"325 YISHUN CTRL, Singapore"


##### Check bad data transforms
There should be 0 bad data transform

In [4]:
resale_df[pd.isnull(resale_df['ADDRESS'])]

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,ADDRESS


##### Get unique addresses needed and rename naming shortcuts for geocode
To lower no. of requests to APIs -> Lower costs to be incurred

In [5]:
unique_address_df = resale_df['ADDRESS'].drop_duplicates().to_frame()
unique_address_df['ADDRESS'] = unique_address_df['ADDRESS'].str.replace('BT', 'BUKIT')
unique_address_df['ADDRESS'] = unique_address_df['ADDRESS'].str.replace("C'WEALTH", "COMMONWEALTH")
unique_address_df['ADDRESS'] = unique_address_df['ADDRESS'].str.replace('JLN', 'JALAN')
unique_address_df['ADDRESS'] = unique_address_df['ADDRESS'].str.replace('NILE', 'NIEL')
unique_address_df

,ADDRESS
0,"172 ANG MO KIO AVE 4, Singapore"
1,"510 ANG MO KIO AVE 8, Singapore"
2,"610 ANG MO KIO AVE 4, Singapore"
3,"474 ANG MO KIO AVE 10, Singapore"
4,"604 ANG MO KIO AVE 5, Singapore"
...,...
481228,"158 ANG MO KIO AVE 4, Singapore"
742369,"340 ANG MO KIO AVE 1, Singapore"
781181,"727 CLEMENTI WEST ST 2, Singapore"
789918,"721 CLEMENTI WEST ST 2, Singapore"


# Geocoding APIs

### MapBox Geocoding

##### Initialise and testing

In [7]:
geolocator_mapbox = MapBox(api_key="pk.eyJ1Ijoia2VhbmVjb2RlcyIsImEiOiJjand2cDJxbmMwYmVlNDhudDNwMWFpbGQ3In0.7wi6WagMsg6t-F3GxB3Cow", timeout=5)
# geocode = lambda query: geolocator_mapbox.geocode("%s, Singapore" % query)
location = geolocator_mapbox.geocode("25 NEW UPP CHANGI RD, Singapore")
print(location.point)
print(location.point.latitude)
location

1 19m 23.5776s N, 103 56m 1.59s E
1.323216


Location(25 New Upper Changi Road, Singapore 462025, Singapore, (1.323216, 103.933775, 0.0))

### Begin Geocoding

In [8]:
mapbox_geocode_df = unique_address_df.copy()
mapbox_geocode = RateLimiter(geolocator_mapbox.geocode, min_delay_seconds=1/5000)
mapbox_geocode_df['geocode'] = mapbox_geocode_df['ADDRESS'].apply(mapbox_geocode)
pd.options.display.max_rows = 10000
mapbox_geocode_df

,ADDRESS,geocode
0,"172 ANG MO KIO AVE 4, Singapore","(172 Ang Mo Kio Avenue 4, Singapore 561172, Si..."
1,"510 ANG MO KIO AVE 8, Singapore","(510 Ang Mo Kio Avenue 8, Singapore 560510, Si..."
2,"610 ANG MO KIO AVE 4, Singapore","(610 Ang Mo Kio Avenue 4, Singapore 560610, Si..."
3,"474 ANG MO KIO AVE 10, Singapore","(474 Ang Mo Kio Avenue 10, Singapore 560474, S..."
4,"604 ANG MO KIO AVE 5, Singapore","(604 Ang Mo Kio Avenue 5, Singapore 560604, Si..."
5,"154 ANG MO KIO AVE 5, Singapore","(154 Ang Mo Kio Avenue 5, Singapore 560154, Si..."
6,"110 ANG MO KIO AVE 4, Singapore","(110 Ang Mo Kio Avenue 4, Singapore 560110, Si..."
7,"445 ANG MO KIO AVE 10, Singapore","(445 Ang Mo Kio Avenue 10, Singapore 560445, S..."
8,"476 ANG MO KIO AVE 10, Singapore","(476 Ang Mo Kio Avenue 10, Singapore 560476, S..."
9,"631 ANG MO KIO AVE 4, Singapore","(631 Ang Mo Kio Avenue 4, Singapore 560631, Si..."


#### Missing results
if things go well, there should be no None returned from MapBox geolocator

In [9]:
mapbox_geocode_df[pd.isnull(mapbox_geocode_df['geocode'])]

,ADDRESS,geocode


##### Bad results
geolocators often return the coordinates outside of Singapore, so we should filter those out and rectify it
if things go well, there should be nothing returned from MapBox geolocator

In [10]:
bad_mapbox_df = mapbox_geocode_df.copy()

bad_mapbox_df.info()
# bad_mapbox_df = bad_mapbox_df[(bad_mapbox_df['geocode']).isin([g for g in bad_mapbox_df['geocode'] if 'Singapore' not in g.raw['place_name']])]
[g for g in bad_mapbox_df['geocode'] if 'Singapore' not in g.raw['place_name']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9317 entries, 0 to 792636
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ADDRESS  9317 non-null   object
 1   geocode  9317 non-null   object
dtypes: object(2)
memory usage: 218.4+ KB


[]

#### Merge unique addresses to original resale_df

In [12]:
merge_address_df = mapbox_geocode_df.copy()
original_resale_df = resale_df.copy()

merge_address_df['ADDRESS'] = merge_address_df['ADDRESS'].str.replace('BUKIT', 'BT')
merge_address_df['ADDRESS'] = merge_address_df['ADDRESS'].str.replace("COMMONWEALTH", "C'WEALTH")
merge_address_df['ADDRESS'] = merge_address_df['ADDRESS'].str.replace('JALAN', 'JLN')
merge_address_df['ADDRESS'] = merge_address_df['ADDRESS'].str.replace('NIEL', 'NILE')

geocoded_resale_df = pd.merge(original_resale_df, merge_address_df, on=['ADDRESS'], how='inner').copy()

geocoded_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,ADDRESS,geocode
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN,"172 ANG MO KIO AVE 4, Singapore","(172 Ang Mo Kio Avenue 4, Singapore 561172, Si..."
1,2012-04,ANG MO KIO,3 ROOM,172,ANG MO KIO AVE 4,06 TO 10,60.0,Improved,1986,302500.0,NaN,"172 ANG MO KIO AVE 4, Singapore","(172 Ang Mo Kio Avenue 4, Singapore 561172, Si..."
2,2012-05,ANG MO KIO,3 ROOM,172,ANG MO KIO AVE 4,06 TO 10,60.0,Improved,1986,295000.0,NaN,"172 ANG MO KIO AVE 4, Singapore","(172 Ang Mo Kio Avenue 4, Singapore 561172, Si..."
3,2012-06,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,04 TO 06,45.0,Improved,1986,259000.0,NaN,"172 ANG MO KIO AVE 4, Singapore","(172 Ang Mo Kio Avenue 4, Singapore 561172, Si..."
4,2012-08,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,263000.0,NaN,"172 ANG MO KIO AVE 4, Singapore","(172 Ang Mo Kio Avenue 4, Singapore 561172, Si..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
831331,2010-07,ANG MO KIO,3 ROOM,340,ANG MO KIO AVE 1,01 TO 03,66.0,Improved,1981,260000.0,NaN,"340 ANG MO KIO AVE 1, Singapore","(340 Ang Mo Kio Avenue 1, Singapore 560340, Si..."
831332,2011-05,ANG MO KIO,3 ROOM,340,ANG MO KIO AVE 1,01 TO 03,66.0,Improved,1981,278000.0,NaN,"340 ANG MO KIO AVE 1, Singapore","(340 Ang Mo Kio Avenue 1, Singapore 560340, Si..."
831333,2011-07,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,355000.0,NaN,"727 CLEMENTI WEST ST 2, Singapore","(727 Clementi West Street 2, Singapore 120727,..."
831334,2011-12,CLEMENTI,3 ROOM,721,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,371000.0,NaN,"721 CLEMENTI WEST ST 2, Singapore","(721 Clementi West Street 2, Singapore 120721,..."


#### Missing results
if things go well, there should be no None returned from MapBox geolocator

In [13]:
geocoded_resale_df[pd.isnull(geocoded_resale_df['geocode'])]

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,ADDRESS,geocode


## Finalise DataFrame for output

In [14]:
latlng_resale_df = geocoded_resale_df.copy()
latlng_resale_df['point'] = geocoded_resale_df['geocode'].apply(lambda loc: tuple(loc.point) if loc else None)
latlng_resale_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(latlng_resale_df['point'].tolist(), index=geocoded_resale_df.index)
latlng_resale_df = latlng_resale_df.drop(['ADDRESS','geocode','altitude','point'], axis=1)
latlng_resale_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,latitude,longitude
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN,1.374573,103.836692
1,2012-04,ANG MO KIO,3 ROOM,172,ANG MO KIO AVE 4,06 TO 10,60.0,Improved,1986,302500.0,NaN,1.374573,103.836692
2,2012-05,ANG MO KIO,3 ROOM,172,ANG MO KIO AVE 4,06 TO 10,60.0,Improved,1986,295000.0,NaN,1.374573,103.836692
3,2012-06,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,04 TO 06,45.0,Improved,1986,259000.0,NaN,1.374573,103.836692
4,2012-08,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,263000.0,NaN,1.374573,103.836692
...,...,...,...,...,...,...,...,...,...,...,...,...,...
831331,2010-07,ANG MO KIO,3 ROOM,340,ANG MO KIO AVE 1,01 TO 03,66.0,Improved,1981,260000.0,NaN,1.364456,103.849468
831332,2011-05,ANG MO KIO,3 ROOM,340,ANG MO KIO AVE 1,01 TO 03,66.0,Improved,1981,278000.0,NaN,1.364456,103.849468
831333,2011-07,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,355000.0,NaN,1.304364,103.763843
831334,2011-12,CLEMENTI,3 ROOM,721,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,371000.0,NaN,1.302703,103.764383


In [15]:
latlng_resale_df.to_csv('./resale.csv', index=False)